In [73]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [74]:
df = pd.read_csv("../hfactory_magic_folders/hi__paris_hackathon/building_energy_efficiency/datasets/test/test_features_sent.csv")

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3135: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [75]:
from copy import deepcopy
df_clean = deepcopy(df)

### Dropping some lines


In [76]:
df_clean.shape 


(494940, 71)

In [77]:
df_clean.shape

(494940, 71)

### Changing Values based on other categorical values

In [78]:
#change the value of the 'balcony_depth' column to 0 for the rows where 'has_balcony' is false
df_clean.loc[df_clean['has_balcony'] == False, 'balcony_depth'] = 0

In [79]:
# Replacing NaN values of the features ['altitude','building_height_ft','building_total_area_sqft'] by the mean
# depending on the building type

df_mean = df_clean.groupby('building_type').mean()[['altitude','building_height_ft','building_total_area_sqft']]

for building_type in df_clean['building_type'].unique():
    df_clean.loc[(df_clean['altitude'].isna()) & (df_clean['building_type'] == building_type), 'altitude'] = df_mean.loc[building_type]['altitude']
    df_clean.loc[(df_clean['building_height_ft'].isna()) & (df_clean['building_type'] == building_type), 'building_height_ft'] = df_mean.loc[building_type]['building_height_ft']
    df_clean.loc[(df_clean['building_total_area_sqft'].isna()) & (df_clean['building_type'] == building_type), 'building_total_area_sqft'] = df_mean.loc[building_type]['building_total_area_sqft']


### Dropping less important features

In [80]:
features = ["nb_commercial_units","nb_gas_meters_commercial","nb_gas_meters_housing","nb_gas_meters_total",
            "nb_housing_units","nb_parking_spaces","nb_power_meters_housing","nb_power_meters_commercial",
            "nb_power_meters_total", "nb_units_total","renewable_energy_sources","building_use_type_code",
            "building_year","consumption_measurement_date",'main_heating_type','nb_meters',
           "outer_wall_materials","post_code","radon_risk_level","roof_material","water_heaters","window_filling_type",
           "window_orientation","main_heat_generators","building_category","clay_risk_level",
            "is_crossing_building","heat_generators","main_heating_type","level_0","water_heaters","main_water_heaters",
            "main_water_heating_type",'building_class']

def remove_features(df,features):        
    # removing features
    df_clean = df.drop(features,axis=1)
    return (df_clean)

df_clean = remove_features(df_clean,features)

In [83]:
df_clean['additional_water_heaters'].replace(np.nan,0,inplace=True)

### Dealing with missing values


In [84]:
def replacing_missing_values(df):
    #Feature Type
    fl_cols = list(df_clean.columns[df_clean.dtypes=="float64"])  # numerical
    int_cols = list(df_clean.columns[df_clean.dtypes=="int64"]) # categorical
    num=fl_cols+int_cols
    categ=list(df_clean.columns[df_clean.dtypes=="object"])
    
    
    for f in fl_cols:
        df[f] = df[f].fillna(df[f].mean())
    
    for n in int_cols:
        df[n] = df[n].fillna(df[n].median())
    
    for c in categ:
        df[c].fillna(df[c].mode()[0], inplace=True) 
    return df    
       

In [85]:
df_clean=replacing_missing_values(df_clean)

In [86]:
categorical_features=list(df_clean.columns[df_clean.dtypes=="object"])


### Label encoders


In [87]:
categ=list(df_clean.columns[df_clean.dtypes=="object"])
def label_encoding(df,categ):
    for c in categ:
        df[c] = df[c].astype(str)
        le = LabelEncoder()
        le.fit(df[c])
        df[c]=le.transform(df[c])
    return df    

In [88]:
df_clean=label_encoding(df_clean,categ)

In [92]:
df_clean.head()

,additional_heat_generators,additional_water_heaters,altitude,area_code,balcony_depth,bearing_wall_material,building_height_ft,building_period,building_total_area_sqft,building_type,...,upper_floor_material,upper_floor_thermal_conductivity,ventilation_type,wall_insulation_type,water_heating_energy_source,water_heating_type,window_frame_material,window_glazing_type,window_heat_retention_factor,window_thermal_conductivity
0,3,0,249.34384,615,0,6,19.685040,1,581.2506,1,...,7,1.749949,13,4,2,1,7,0,1.067688,21.349539
1,3,0,104.98688,304,0,22,28.215224,1,721.1813,1,...,3,1.665767,10,7,2,1,6,0,1.149140,13.963716
2,3,0,72.17848,204,0,6,25.918636,0,699.6535,1,...,3,1.770545,13,7,3,1,6,0,1.084204,14.649374
3,21,0,800.52496,858,0,6,15.748032,2,1248.6124,1,...,5,1.790967,3,4,2,1,7,0,1.067688,15.347292
4,27,0,1010.49872,671,0,18,0.000000,4,1345.4875,1,...,12,1.790967,10,4,2,1,6,0,3.592767,17.512631


## Importing the model

In [89]:
df_clean.shape

(494940, 39)

In [45]:
import pickle

In [47]:
with open('../my_work/xgboost_model_bis.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [48]:
y_pred = loaded_model.predict(df_clean)

In [49]:
y_pred.shape

(494940,)

In [72]:
df_clean['building_year']

0         1965.0
1         1850.0
2         1966.0
3         2001.0
4         2008.0
           ...  
494935    2013.0
494936    2017.0
494937    1978.0
494938    1999.0
494939    1998.0
Name: building_year, Length: 494940, dtype: float64

In [54]:
df

,level_0,additional_heat_generators,additional_water_heaters,altitude,area_code,balcony_depth,bearing_wall_material,building_category,building_class,building_height_ft,...,wall_insulation_type,water_heaters,water_heating_energy_source,water_heating_type,window_filling_type,window_frame_material,window_glazing_type,window_heat_retention_factor,window_orientation,window_thermal_conductivity
0,1371914,[],NaN,249.34384,615,NaN,BRICKS,[],[],19.685040,...,internal,[electric storage tank],electricity,individual,dry air,wood,double glazing,1.067688,"[north,sud]",21.349539
1,618756,[],NaN,104.98688,304,NaN,STONE,[],[],28.215224,...,non insulated,[electric storage tank],electricity,individual,argon or krypton,pvc,double glazing,1.149140,[west],13.963716
2,871032,[],NaN,72.17848,204,NaN,BRICKS,[individual house],[individual],25.918636,...,non insulated,[indeterminate gas boiler],gas,individual,dry air,pvc,double glazing,1.084204,"[north,sud]",14.649374
3,1430640,[oil stove or insert],NaN,800.52496,858,NaN,BRICKS,[individual house],[individual],15.748032,...,internal,[electric storage tank],electricity,individual,dry air,wood,double glazing,1.067688,"[est,west]",15.347292
4,86306,[wood stove or insert],NaN,1010.49872,671,NaN,GRITSTONE,[individual house],[individual],0.000000,...,internal,[electric storage tank],electricity,individual,dry air,pvc,double glazing,3.592767,[],17.512631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494935,845834,[],NaN,NaN,467,NaN,OTHER,[],[],NaN,...,internal,[electric storage tank],electricity,individual,dry air,wood,double glazing,3.887723,[],17.512631
494936,279675,[],NaN,377.29660,748,NaN,CONCRETE,[],[],15.748032,...,internal,[electric storage tank],electricity,individual,argon or krypton,pvc,double glazing,1.084204,[north],13.963716
494937,811813,[],NaN,830.05252,838,NaN,CONCRETE,[condo],[2 to 11],36.089240,...,internal,[indeterminate gas boiler],gas,individual,argon or krypton,pvc,double glazing,1.034308,[est],8.947385
494938,369033,[wood stove or insert],NaN,288.71392,752,NaN,AGGLOMERATE,[individual house],[individual],14.763780,...,internal,[electric storage tank],electricity,individual,dry air,pvc,double glazing,1.084204,"[est,west]",13.963716


In [69]:
submission = pd.concat([df['level_0'],pd.Series(y_pred)],axis = 1,keys=['level_0', 'energy_consumption_per_annum'])

In [70]:
submission.head()

,level_0,energy_consumption_per_annum
0,1371914,161.655136
1,618756,163.235092
2,871032,251.794342
3,1430640,169.683533
4,86306,181.097916


In [71]:
submission.to_csv("../my_work/first_attempt.csv",index = False)